*please execute the cell below before starting the tutorial by selecting it and pressing Ctrl+Enter*

In [ ]:
from aiida.orm import Dict, load_node
from aiida.engine import submit

# Input generator code

On Monday, you learned about simplified input file for the FLEUR code and inpgen code. This code generates an `inp.xml` input-file for FLEUR using the simplified input. In AiiDA, a `Code` object representing input generator has to have a similar behaviour: it has to generate a `FleurinpData` from a list of simplified parameters. The code class that implements this is called `FleurinputgenCalculation` and in this tutorial we are going to learn how to use it in AiiDA interface.

First things first - let us get familiar with inputs of the FleurinputgenCalculation.

## Inputs of the input calculation

Input generator input nodes has only 4 main input nodes that are listed in the table below:

|name|type|describtion| required |
|:---:|:---:|:---------:|:---:|
|code | Code | Inpgen code| yes |
|structure | StructureData | Structure data node| yes |
|parameters | Dict | FLAPW parameters| no |
|settings | Dict | special settings | no |

**code** is a `Code` representing inpgen inself in the database. 

Simplified input file is represented as a pair of nodes: **structure** and **parameters**. **structure** is a `StructureData` node to be inserted into the simpified input file. **parameters** is a nested dictionary containing nested name: value pairs and representing all other parameters of the simplifier input file. For instance, if one wants to add a line

    &kpt
      div1=24   div2=20   div3=1   tkb=0.001 /

in the simplified input file, it is necessary to append

    'kpt': {
        'div1': 24,
        'div2' : 20,
        'div3' : 1
        }

to the **parameters** dictionary. In some cases the simplified input file does not have a name for some nested values, such as for SOC. In this case you can refer to the list of all supported keys:

    'input': ['film', 'cartesian', 'cal_symm', 'checkinp',
              'symor', 'oldfleur'],
    'lattice': ['latsys', 'a0', 'a', 'b', 'c', 'alpha',
                'beta', 'gamma'],
    'atom': ['id', 'z', 'rmt', 'dx', 'jri', 'lmax',
             'lnonsph', 'ncst', 'econfig', 'bmu', 'lo',
             'element', 'name'],
    'comp': ['jspins', 'frcor', 'ctail', 'kcrel', 'gmax',
             'gmaxxc', 'kmax'],
    'exco': ['xctyp', 'relxc'],
    'film': ['dvac', 'dtild'],
    'soc': ['theta', 'phi'],
    'qss': ['x', 'y', 'z'],
    'kpt': ['nkpt', 'kpts', 'div1', 'div2', 'div3',
            'tkb', 'tria'],
    'title': {}
                        
and find that to correctly specify SOC ``theta`` and ``phi`` values, you need to append

    'soc': {
        'theta': 0.7,
        'phi' : 0.26
        }

to the **parameters** dictionary.

**settings** is used to modify a list of files to be copied to/from/between calculation folders and to append command options to the executable. We will not cover this in the tutorial.


## Inpgen code submission

To sumbit an inpgen calculation, we need to import `FleurinputgenCalculation` class first.

In [ ]:
from aiida_fleur.calculation.fleurinputgen import FleurinputgenCalculation

and prepare input nodes. Configured and ready-to-use inpgen code object is already stored in your database under the number pk=xxx. Now you can simple load it from the database:

In [ ]:
code = load_node(xxx)

We are going to use a structure that you have created in tutorial 1. Please, find a pk of an Fe monolayer structure and load in via the same command:

In [ ]:
structure = load_node(xxx)

Finally, we need to setup a **parameters** dictionary. For now we are going to set k-mesh and kmax only:

In [ ]:
parameters = Dict(dict={
    'comp': {
        'kmax': 3.85,
        },
    'kpt': {
        'div1': 12,
        'div2' : 10,
        'div3' : 1
        }})

In [ ]:
res = submit(FleurinputgenCalculation, code=code, structure=structure, parameters=parameters)

The calculation has to finish fast - you will probably not be able to catch it in not 'FINISHED' state.
You can explore it's outputs: